# Preparing the expression background - Gtex toil dataset

The dataset will be re-structured in the directory `../input/gene_expression/[DATASETNAME]`. The code internally will generate one `metadata_[DATASTNAME].csv` inside that folder. This file is a table with two columns: tissue (name of the tissue) and nsamples (number of samples), the names used for tissues inside this table will be the ones used to name the individual tissue folders containing the expression data files and to build the job array list. In addition, it will create one folder per tissue inside `../input/gene_expression/[DATASETNAME]/subgroups` , inside each tissue folder two files will be created: `[tissue].RDS` and `[tissue].csv` with the same counts matrix stored in the two different formats.  

In summary, the folder will look like this:

- `../input/gene_expression/[DATASETNAME]`   
    - `metadata_[DATASTNAME].csv` dataset metadata file generated automatically when splitting the data into tissues
    - `genes_[DATASETNAME].csv` single-column file generated automatically with gene ids of all genes in universe
    - `subgroups`   
        - `tissue_1`   
            - `tissue_1.csv` expression matrix table in CSV format   
        - `tissue_2`    
            - `tissue_2.csv`
         
**Definition of the gene universe**

The file `genes_[DATASETNAME].csv` contains the list of genes in the gene universe of this dataset. This list contains the genes that have at least `min_cts` in at least `min_sam_cts` samples. By default, these parameters are set to 3 counts in at least 1 sample. These filters are part of the changes implemented in version 2.2 and generate datasets with the tag `gfilter`.

In [1]:
suppressPackageStartupMessages({
    library(dplyr)
    library(plyr)
    library(biomaRt)
    library(data.table)
    library(furrr)
})
source('../funcs/misc.R')
options(stringsAsFactors=FALSE)

## PDxN requirements

In [2]:
# Dataset name
dsname<-"gtextoil_gfilter" # Dataset name (will be used in the config file) 

# Filtering parameters
min_cts<-3 # Minimum number of gene counts
min_sam_cts<-1 # Minimum number of samples with min_cts
min_sam_tissue<-10 # Minimum number of samples per tissue

In [3]:
# Directory structure - DO NOT CHANGE
output_dir <- file.path("../../input/gene_expression",dsname)
metadata_file <- file.path(output_dir,paste0("metadata_",dsname,".csv"))
gene_file <- file.path(output_dir,paste0("genes_",dsname,".csv"))
create_directory(output_dir)
message("Creating dataset directory at ",output_dir)

Creating dataset directory at ../../input/gene_expression/gtextoil_gfilter



## Dataset-specific inputs

In [3]:
# Dataset-specific inputs
indir<-"~/projects/pdxn_2.0/data/background/gtex_toil" # Directory with raw data
ctsfile<-"gtex_RSEM_Hugo_norm_count" # File with gene counts
genefile<-"gtex_hugo_gencode_good_hg38_v23comp_probemap" # File with gene metadata
metafile<-"GTEX_phenotype" # File with sample metadata

In [4]:
# Load dataset
cts <- fread(file.path(indir,ctsfile))
idmap<-fread(file.path(indir,genefile))
meta <- fread(file.path(indir,metafile))

## Process original files 

This section takes the raw files downloaded from the GTex website and re-formats the counts, gene and sample metadata to:
1. Have accurate and computer-readable annotations of the tissue of origin 
2. Map the ENSEMBL gene IDs to entrez gene IDs.
3. Process the gene counts in the case of multiple ENSEMBL IDs mapping to a single entrez IDs by taking the median counts.
4. Generate a global count table of Entrez gene IDs x all samples (with valid tissue annotations) for the next step

#### Re-shape metadata table

In [5]:
# Filter samples 
meta <- meta %>%
             dplyr::rename("primary_site"="_primary_site",
                           "gender"="_gender",
                           "patient"="_patient",
                           "cohort"="_cohort",
                           "body_site"="body_site_detail (SMTSD)") %>%
             filter(primary_site!="<not provided>") %>%
             mutate(primary_site=gsub(" ","",primary_site),
                    body_site=gsub(" | - ","",body_site) %>% gsub("\\(","-",.) %>% gsub("\\)","",.)) %>%
             mutate(tissue = paste(primary_site,
                                   body_site,
                                   sep="_"))  %>%
             mutate(tissue = ifelse(primary_site==body_site,primary_site,tissue))

In [6]:
metafile.out<-paste0(indir,"/",metafile,"_processed.tsv")
write.table(meta,file=metafile.out,quote=FALSE,sep="\t",row.names=FALSE,col.names = TRUE)

#### Map gene symbols to entrez IDs

In [7]:
# Map hugo gene names to entrez ids
ensembl <- useEnsembl(biomart = "ensembl", dataset = "hsapiens_gene_ensembl") 
attributes <- c("hgnc_symbol", "entrezgene_id")
gene_mappings <- getBM(attributes = attributes, 
                       filters = "hgnc_symbol", 
                       values = idmap$id, 
                       mart = ensembl) %>%
                 filter(!is.na(entrezgene_id))
write.table(gene_mappings,file=file.path(indir,"gtex_RSEM_Hugo_norm_count_gene_mappings.csv"),row.names=F,quote=F,sep=",")

In [8]:
dim(gene_mappings)
head(gene_mappings)

[1] 23441     2

,hgnc_symbol,entrezgene_id
,<chr>,<int>
1,AAAS,8086
2,AAGAB,79719
3,ABAT,18
4,ABHD14A-ACY1,100526760
5,ABI1,10006
6,ABI2,10152


In [9]:
# Collapse multiple probes to their corresponding entrez gene id by taking gene-level median values
cts.proc <- cts %>%
            dplyr::rename('hgnc_symbol'='sample') %>%
            inner_join(.,gene_mappings) %>% 
            as.data.frame() %>%
            dplyr::select(-hgnc_symbol) %>%
            group_by(entrezgene_id) %>%
            dplyr::summarise_all(.,median)

Joining with `by = join_by(hgnc_symbol)`


In [ ]:
# Rename rows to entrez gene ids
cts.proc <- cts.proc %>%
            tibble::column_to_rownames('entrezgene_id')
write.table(cts.proc,file=file.path(indir,"gtex_RSEM_Hugo_norm_count_entrez_mapped.csv"),
            row.names=TRUE,quote=FALSE)

In [ ]:
cts.proc[1:10,1:10]

## Split dataset by tissue

In this section we take the processed input files (counts and metadata) and split the global count matrix into different tissues to create the dataset structure required for PDxN:

- `../input/gene_expression/[DATASETNAME]`   
    - `metadata_[DATASTNAME].csv` dataset metadata file generated automatically when splitting the data into tissues
    - `genes_[DATASETNAME].csv` single-column file generated automatically with gene ids of all genes in universe
    - `subgroups`   
        - `tissue_1`   
            - `tissue_1.csv` expression matrix table in CSV format containing samples from tissue 1  
        - `tissue_2`    
            - `tissue_2.csv`expression matrix table in CSV format containing samples from tissue 2

In [ ]:
cts.proc<-read.table(file.path(indir,"gtex_RSEM_Hugo_norm_count_entrez_mapped.csv")) 
meta<-read.table(paste0(indir,"/",metafile,"_processed.tsv"),header = T)

In [ ]:
# Filter samples without tissue annotation removed at the metadata stage
meta<-meta %>%
        mutate(Sample_name = gsub("-",".",Sample)) %>%
        filter(Sample_name %in% colnames(cts.proc)) 
        
cts.proc<-cts.proc[,meta$Sample_name] # Filter counts to match samples in metadata
message("Discarded ",ncol(cts)-ncol(cts.proc)," samples due to missing sample info")

In [ ]:
# Build list of count matrices by tissue
cts.df <- meta %>%
            dplyr::select(tissue,Sample_name) %>%
            group_by(tissue) %>%
            tidyr::nest(samples=c(Sample_name)) %>%
            ungroup() %>%
            mutate(data=purrr::map(samples,function(s,...){ cts.proc[,s[[1]]] } )) %>%
            mutate(nsamples=length(samples))%>%
            filter(nsamples>=min_sam_tissue) %>%
            dplyr::select(tissue,data)
cts.list <- cts.df$data
names(cts.list) <- cts.df$tissue

In [ ]:
# Filter gene expression matrices 
cts.list.filt <- filter_tissue_expr_list(cts_list = cts.list,
                                       min_counts = min_cts,
                                       min_samples = min_sam_cts,
                                       gu_file = gene_file)
names(cts.list.filt$filtered_cts) <- names(cts.list)

In [ ]:
# Create tissue files  - the function writes the files internally 
res <- tissue_list_to_dirs(cts_list = cts.list.filt$filtered_cts,
                           output_dir = output_dir,
                           meta_file = metadata_file)

In [ ]:
# Verify that the matrices have the correct dimensions and the splitting was successful
str(res)